# Nonlinear analysis -Assignment 1

Due: 27.09.2023 - Jérémie Engler & Laure Toullier

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('bmh')
import math

np.set_printoptions(precision=3, suppress=True)

Params = plt.rcParams
Params['figure.figsize'] = (14, 7) 

### Exercise 2

#### Geometry

In [2]:
#Geometric data for the truss:
E=200000

DDL=7  #degree of freedom
a=3000 #mm
b=4000 #mm
support=[0,7] #number of the nodes where there is a support

In [3]:
#Load definition
load=150*10**3 #N, value of the applied load
P=np.zeros(2*DDL).reshape(DDL,2) 
theta_P=-90 #°, inclination of the load regarding the x-axis

    #Application of the load at the corresponding nodes:

P[1]=[load*np.cos(np.radians(theta_P)),load*np.sin(np.radians(theta_P))]
P[3]=[load*np.cos(np.radians(theta_P)),load*np.sin(np.radians(theta_P))]
P[5]=[load*np.cos(np.radians(theta_P)),load*np.sin(np.radians(theta_P))]

In [4]:
#Generate the nodes of the truss and their location (if 1st node at (0,0))

nodes=np.zeros(DDL*2).reshape(DDL,2)

for i in range(DDL):
    nodes[i,0]=a*i
    if i%2==0:
        nodes[i,1]=0
    else:
        nodes[i,1]=b

print(nodes)

[[    0.     0.]
 [ 3000.  4000.]
 [ 6000.     0.]
 [ 9000.  4000.]
 [12000.     0.]
 [15000.  4000.]
 [18000.     0.]]


In [5]:
#Connectivity: connect nodes with bar element

connectivity=[]

for i in range(DDL):
    if i<(DDL-2):
        connectivity.append([i+1, i+2])
        connectivity.append([i+1, i+3])
    if i==DDL-2:
        connectivity.append([i+1, i+2])
        
Connectivity=np.array(connectivity)
    
print(Connectivity)

[[1 2]
 [1 3]
 [2 3]
 [2 4]
 [3 4]
 [3 5]
 [4 5]
 [4 6]
 [5 6]
 [5 7]
 [6 7]]


In [6]:
# Area: area of each bar of the truss

A=10*10**3 #mm²     #members area equal in the entire truss
nb_members=int(np.size(Connectivity)/2)
area=np.zeros(nb_members)

for i in range(np.size(area)):
    area[i]=A
    
area

array([10000., 10000., 10000., 10000., 10000., 10000., 10000., 10000.,
       10000., 10000., 10000.])

In [7]:
#Length of each bar element of the truss, from points' coordinates of each bar.

length=[]

for i in range(nb_members):
    x1=nodes[Connectivity[i,0]-1,0]      #For each node at the end of each bar, we get its location in x and y.
    y1=nodes[Connectivity[i,0]-1,1]
    x2=nodes[Connectivity[i,1]-1,0]
    y2=nodes[Connectivity[i,1]-1,1]
    
    x=x2-x1
    y=y2-y1
    length.append(np.sqrt(x**2+y**2))

length

[5000.0,
 6000.0,
 5000.0,
 6000.0,
 5000.0,
 6000.0,
 5000.0,
 6000.0,
 5000.0,
 6000.0,
 5000.0]

#### Stiffness

In [8]:
#Local stiffness matrix for each bar element of the truss

k_local=E*area/length
k_local

NameError: name 'E' is not defined

In [ ]:
# Unit stiffness matrix of bar in local coordinate system
K_unit = np.array([[1, 0, -1, 0], 
                   [0, 0,  0, 0], 
                   [-1, 0, 1, 0], 
                   [0, 0, 0, 0]])

In [ ]:
#Angle of each bar regarding the x-axis

angles=[]

for i in range(nb_members):
        
    x1=nodes[Connectivity[i,0]-1,0]      #For each node at the extremity of each bar, we get its location in x and y.
    y1=nodes[Connectivity[i,0]-1,1]
    x2=nodes[Connectivity[i,1]-1,0]
    y2=nodes[Connectivity[i,1]-1,1]
    
    if np.isclose(y1, y2):
        if np.isclose(x1, x2):
            angles.append(90)
        else:
            angles.append(0)
    else:
        theta=np.arcsin((y2-y1)/length[i]) #calculate the angle between the x-axis and the bar element
        angles.append(math.degrees(theta))
    
print(angles)


In [ ]:
#Transformation matrix for each bar element of the truss

c=[]
s=[]

for i in range(nb_members):
    c.append(np.cos(np.radians(angles[i])))
    s.append(np.sin(np.radians(angles[i])))
    
T=np.zeros((nb_members,4,4))

for i in range(nb_members):
    T[i,0,0]=c[i]
    T[i,0,1]=s[i]
    T[i,1,0]=-s[i]
    T[i,1,1]=c[i]
    T[i,2,2]=c[i]
    T[i,2,3]=s[i]
    T[i,3,2]=-s[i]
    T[i,3,3]=c[i]

T

In [ ]:
#Stiffness for each bar in global coordinates system

k_global=[]

for i in range(nb_members):
    k=k_local[i]*T[i].T@K_unit@T[i] 
    k_global.append(k)

print(k_global)

In [ ]:
#Stiffness in global system

K_globalCS_bar=np.zeros((nb_members, DDL*2,DDL*2))
K_global=np.zeros((DDL*2,DDL*2))
k_global = np.array(k_global)

for i, c in enumerate(Connectivity):       
    #we build all the global stiffness matrix for each bar in the entire system and sum them.
    u, v = c
    K_global[2*u-2:2*u,2*u-2:2*u] += k_global[i, 0:2, 0:2]
    K_global[2*u-2:2*u,2*v-2:2*v] += k_global[i, 0:2, 2:4]
    K_global[2*v-2:2*v,2*u-2:2*u] += k_global[i, 2:4, 0:2]
    K_global[2*v-2:2*v,2*v-2:2*v] += k_global[i, 2:4, 2:4]
    
    #global stiffness matrix for each bar in the entire system 
    K_globalCS_bar[i, 2*u-2:2*u,2*u-2:2*u] = k_global[i, 0:2, 0:2]
    K_globalCS_bar[i, 2*u-2:2*u,2*v-2:2*v] = k_global[i, 0:2, 2:4]
    K_globalCS_bar[i, 2*v-2:2*v,2*u-2:2*u] = k_global[i, 2:4, 0:2]
    K_globalCS_bar[i, 2*v-2:2*v,2*v-2:2*v] = k_global[i, 2:4, 2:4]

print(K_global)

#### Equilibrium of the entire system : K*U=R

In [ ]:
#List of nodes with a load applied on it

nodes_with_load=[]

for i, c in enumerate(P):
    x,y = c
    if x!=0 or y!=0:
        nodes_with_load.append(i)

nodes_with_load=np.array(nodes_with_load)

#Calculate the displacements of the nodes with a load applied

F_red=np.zeros(DDL*2-2*len(support))

for i in range(DDL):
    if i in nodes_with_load:
        F_red[i*2-2]=P[i,0]
        F_red[i*2-1]=P[i,1]  
        
a=2*(DDL-len(support)) 
K_red=np.zeros((a,a)) #reduced stiffness matrix for the nodes where there are displacements
K_red=K_global[2:a+2, 2:a+2]

u_red=np.linalg.inv(K_red)@F_red #reduced vector displacements for the nodes concerned by displacements

u_vec=np.zeros(DDL*2) #global vector for displacements
u_vec[2:DDL*2-2]=u_red[:]

print(u_vec) 


In [ ]:
#Support reactions
F = K_global @ u_vec #N
print(F)

In [ ]:
#Axial forces in each bar of the truss

F_bar=[]
F_bar_reduced=np.zeros((nb_members,4))
F_local_CS=[]
axial_load=[]

for i in range(nb_members):
    F_bar.append(K_globalCS_bar[i,:,:]@u_vec)   #forces applied at each extermity of each bar in global coordinates in global system
F_bar=np.array(F_bar)

for i in range(nb_members): #forces applied at the extremity of each bar in global coordinates in local system
    index_1=Connectivity[i,0] #value of 1st node of the bar
    index_2=Connectivity[i,1] #value of 2nd node of the bar
    F_bar_reduced[i,0]=F_bar[i,(index_1-1)*2] 
    F_bar_reduced[i,1]=F_bar[i,(index_1)*2-1]
    F_bar_reduced[i,2]=F_bar[i,(index_2-1)*2]
    F_bar_reduced[i,3]=F_bar[i,index_2*2-1]     

for i in range(nb_members): #forces applied at the extremity of each bar in local coordinates in local system
    F_local_CS.append(T[i]@F_bar_reduced[i])   
F_local_CS=np.array(F_local_CS)

for i in range(nb_members): #axial load in each bar in N
    axial_load.append(F_local_CS[i,2])
    
axial_load=np.array(axial_load)
print(axial_load)

In [ ]:
#Conclusion - Results - Support reactions and axial loads

nodes_x=[]
nodes_y=[]

for i in range(DDL):
    nodes_x.append(nodes[i,0])
    nodes_y.append(nodes[i,1])
    

x=np.linspace(0,18000,3000)

nodes_xx=[]
nodes_yy=[]

for u, v in Connectivity:
    x1=nodes[u-1,0]
    x2=nodes[v-1,0]
    y1=nodes[u-1,1]
    y2=nodes[v-1,1]
    nodes_xx.append([x1,x2])
    nodes_yy.append([y1,y2])
nodes_xx = np.array(nodes_xx)
nodes_yy = np.array(nodes_yy)


# Displacements
u_vec_array=u_vec.reshape((DDL,2))
f = 100
nodes_displ = nodes + u_vec_array #list of displaced nodes
nodes_displ_exaggerated = nodes + u_vec_array * f

nodes_displ_x=[]
nodes_displ_y=[]
for dispx, dispy in nodes_displ:
    nodes_displ_x.append(dispx)
    nodes_displ_y.append(dispy)

nodes_displ_x_ex=[]
nodes_displ_y_ex=[]
for u, v in Connectivity:
    x1=nodes_displ_exaggerated[u-1,0]
    x2=nodes_displ_exaggerated[v-1,0]
    y1=nodes_displ_exaggerated[u-1,1]
    y2=nodes_displ_exaggerated[v-1,1]
    nodes_displ_x_ex.append([x1,x2])
    nodes_displ_y_ex.append([y1,y2])

print(nodes_xx)

compression_label=False
tension_label=False
displaced_label=False
for nx, ny, dx, dy, axload in zip(nodes_xx, nodes_yy, nodes_displ_x_ex, nodes_displ_y_ex, axial_load):

    # Graph legend
    if axload > 0:
        color='red'
        label='tension' if tension_label is False else None
        tension_label=True
    else:
        color='blue'
        label='compression' if compression_label is False else None
        compression_label=True

    plt.text(
        x=nx.mean(),
        y=ny.mean(),
        s=f"N={axload:+.0f} N",  # :.2f veut dire deux chiffres après la virgule
        bbox=dict(facecolor='white', edgecolor=color, boxstyle='round, pad=1') # optionnel, elève ça si tu veux
    )
    
    # plot the bars
    plt.plot(nx, ny, color=color, linewidth=2, label=label)

    # displaced bars
    plt.plot(dx, dy, '-.', color='k', linewidth=1, label='displaced truss' if displaced_label is False else None)
    displaced_label = True
    plt.title(f"Displacements are exaggerated by a factor of {f:.0f}")

# Forces applied on the truss and support reactions
force_label = False
for (nx, ny), fx, fy in zip(nodes, F[::2], F[1::2]):
    facteur = 0.01
    if not (np.isclose(fx, 0.0) and np.isclose(fy, 0.0)):
        plt.arrow(
            x=nx - fx*facteur,
            y=ny - fy*facteur,
            dx=fx*facteur,
            dy=fy*facteur,
            color='green', width=100,
            length_includes_head=True,
            label='Loads in N' if force_label is False else None
        )
        force_label = True
        
        #text for loads and support reactions values
        plt.text(
            x=nx - 0.5*fx*facteur,
            y=ny - 0.5*fy*facteur,
            s=f"Fx={fx:.0f} N\nFy={fy:.0f} N",  # :.0f no decimal
            bbox=dict(facecolor='white', edgecolor='g', boxstyle='round, pad=1') 
        )


# Displaced nodes
plt.plot(*nodes_displ_exaggerated.T, 'x', markersize=10, label='displaced nodes', color='Green')  
# Nodes not displaced
plt.plot(*nodes.T, 'o', mfc='w', mew=2, mec='k')
plt.tight_layout()

plt.legend()

In [ ]:
#Conclusion - Results - Displacement


compression_label=False
tension_label=False
displaced_label=False
for nx, ny, dx, dy, axload in zip(nodes_xx, nodes_yy, nodes_displ_x_ex, nodes_displ_y_ex, axial_load):

    # Graph legend
    if axload > 0:
        color='red'
        label='tension' if tension_label is False else None
        tension_label=True
    else:
        color='blue'
        label='compression' if compression_label is False else None
        compression_label=True

    # plot the bars
    plt.plot(nx, ny, color=color, linewidth=2, label=label)

    # displaced bars
    plt.plot(dx, dy, '-.', color='k', linewidth=1, label='displaced truss' if displaced_label is False else None)
    displaced_label = True
    plt.title(f"Displacements are exaggerated by a factor of {f:.0f}")

# Forces applied on the truss and support reactions

force_label = False
for (nx, ny), fx, fy, nx, ny, dx, dy in zip(nodes, F[::2], F[1::2],
                                    nodes_displ.flatten()[::2], nodes_displ.flatten()[1::2],
                                    u_vec[::2], u_vec[1::2]):
    facteur = 0.01
    if not (np.isclose(fx, 0.0) and np.isclose(fy, 0.0)):
        plt.arrow(
            x=nx - fx*facteur,
            y=ny - fy*facteur,
            dx=fx*facteur,
            dy=fy*facteur,
            color='green', width=100,
            length_includes_head=True,
            label='Loads in N' if force_label is False else None
        )
        force_label = True

    #text for displaced nodes (dx,dy)
    plt.text(
        x=nx,
        y=ny,
        s=f"dx={dx:+.3f} mm\ndy={dy:+.3f} mm",  
        bbox=dict(facecolor='white', edgecolor=color, boxstyle='round, pad=1') 
    )

# Displaced nodes
plt.plot(*nodes_displ_exaggerated.T, 'x', markersize=10, label='displaced nodes', color='Green')  
# Nodes not displaced
plt.plot(*nodes.T, 'o', mfc='w', mew=2, mec='k')
plt.tight_layout()

plt.legend()